# Clustering Crypto

In [20]:
# Initial imports
import pandas as pd
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import hvplot.pandas
import plotly.figure_factory as ff

### Deliverable 1: Preprocessing the Data for PCA

In [21]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv("crypto_data.csv")
crypto_df.sample(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1092,LTZ,Litecoinz,Equihash,True,PoW,NaN,84000000
770,SMART,SmartCash,Keccak,True,PoW,2.241009e+09,5000000000
1140,BEET,Beetle Coin,XEVAN,True,PoW/PoS,1.989381e+08,500000000
227,PTC,PesetaCoin,Scrypt,True,PoW,1.375177e+08,166386000
716,IEC,IvugeoEvolutionCoin,Scrypt,True,PoW,NaN,100000000
280,EQM,Equilibrium Coin,SHA-256,True,PoW/PoS,NaN,0
891,SGL,Sigil,NeoScrypt,False,PoW,0.000000e+00,50000000
1101,BITM,BitMoney,XEVAN,True,Pos,2.085159e+08,70000000000
1039,ABS,Absolute Coin,Lyra2REv2,True,PoW/PoS,1.333262e+07,52500000
102,GLX,GalaxyCoin,Scrypt,True,PoW/PoS,NaN,538214400


In [22]:
# Keep all the cryptocurrencies that are being traded.
traded_df = crypto_df.loc[crypto_df["IsTrading"] == True]
traded_df.set_index('Unnamed: 0', inplace=True)
traded_df.index.name = None
traded_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [23]:
# Keep all the cryptocurrencies that have a working algorithm.
algorithm_df = traded_df[traded_df["Algorithm"].notnull()]
algorithm_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [24]:
# Remove the "IsTrading" column. 
removed_df = algorithm_df.drop(["IsTrading"], axis=1)
removed_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [25]:
# Remove rows that have at least 1 null value.
null_rows_df = removed_df.dropna()
null_rows_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [26]:
# Keep the rows where coins are mined.
mined_df = null_rows_df[null_rows_df["TotalCoinsMined"] > 0]
mined_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [27]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = mined_df[["CoinName"]]
crypto_names_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [28]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
clean_df = mined_df.drop(["CoinName"], axis=1)
clean_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [29]:
# Use get_dummies() to create variables for text features.
## X is variable
## Features = algorithms and prooftype
X = pd.get_dummies(clean_df, columns=["Algorithm", "ProofType"])
X.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# Standardize the data with StandardScaler()
scaled = StandardScaler().fit_transform(X)
scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [31]:
# Using PCA to reduce dimension to three principal components
## components = 3
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(scaled)
crypto_pca

array([[-0.34318136,  0.96140864, -0.4538216 ],
       [-0.32652399,  0.9614906 , -0.45418825],
       [ 2.31215794,  1.53044842, -0.55264066],
       ...,
       [ 0.33857269, -2.27053713,  0.35475885],
       [-0.13932505, -1.95792035,  0.21035985],
       [-0.30160163,  0.71030529, -0.19352834]])

In [32]:
# Create a DataFrame with the three principal components
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=clean_df.index
)
pca_df.head(10)

,PC 1,PC 2,PC 3
42,-0.343181,0.961409,-0.453822
404,-0.326524,0.961491,-0.454188
1337,2.312158,1.530448,-0.552641
BTC,-0.134403,-1.239060,0.160254
ETH,-0.144010,-1.948152,0.369949
LTC,-0.172784,-1.132511,0.011210
DASH,-0.387570,1.252664,-0.404355
XMR,-0.144716,-2.169992,0.314505
ETC,-0.142452,-1.948261,0.369933
ZEC,-0.139324,-1.957920,0.210360


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [33]:
# Create an elbow curve to find the best value for K.
# Create variables
inertia = []
k = list(range(1,11))

# For loop using inertia
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)
    
# Elbow curve code
elbow_curve = {"k": k, "inertia": inertia}
elbow_curve_df = pd.DataFrame(elbow_curve)
elbow_curve_df.hvplot.line(x="k", y="inertia", xticks=k, title="The Crypto Elbow Curve")

C:\Users\mjrm_\anaconda3\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [34]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)
predictions

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,

In [35]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features
# Concatentate the crypto_df and pcs_df DataFrames on the same columns
# Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies
# Add a new column, "Class" to the clustered_df DataFrame that holds the predictions
clusters_df = pd.DataFrame({
    "Algorithm": clean_df["Algorithm"],
    "ProofType": clean_df["ProofType"],
    "TotalCoinsMined": clean_df["TotalCoinsMined"],
    "TotalCoinSupply": clean_df["TotalCoinSupply"],
    "PC 1": pca_df["PC 1"],
    "PC 2": pca_df["PC 2"],
    "PC 3": pca_df["PC 3"],
    "CoinName": crypto_names_df["CoinName"],
    "Class": predictions
})

# Print the shape of the clustered_df
print(clusters_df.shape)
clusters_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.343181,0.961409,-0.453822,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.326524,0.961491,-0.454188,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.312158,1.530448,-0.552641,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.134403,-1.239060,0.160254,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.144010,-1.948152,0.369949,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.172784,-1.132511,0.011210,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.387570,1.252664,-0.404355,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.144716,-2.169992,0.314505,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.142452,-1.948261,0.369933,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.139324,-1.957920,0.210360,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [36]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clusters_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_data=["Algorithm"],
    hover_name="CoinName",
    color="Class",
    symbol="Class",
    width=800
)

fig.update_layout(legend=dict(x=0,y=1))
fig.show

<bound method BaseFigure.show of Figure({
    'data': [{'customdata': array([['Scrypt'],
                                   ['Scrypt'],
                                   ['X13'],
                                   ...,
                                   ['SHA-256'],
                                   ['Scrypt'],
                                   ['Scrypt']], dtype=object),
              'hovertemplate': ('<b>%{hovertext}</b><br><br>Cla' ... '{customdata[0]}<extra></extra>'),
              'hovertext': array(['42 Coin', '404Coin', 'EliteCoin', ..., 'ZEPHYR', 'Gapcoin',
                                  'BitcoinPlus'], dtype=object),
              'legendgroup': '1',
              'marker': {'color': array([1, 1, 1, ..., 1, 1, 1]),
                         'coloraxis': 'coloraxis',
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '1',
              'scene': 'scene',
              'showlegend': True,
              'type': 'scatter3d',


In [37]:
# Create a table with tradable cryptocurrencies.
columns = ["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"]
clusters_df.hvplot.table(columns)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [38]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {clusters_df.CoinName.size} tradeable cryptocurrencies.")

There are 532 tradeable cryptocurrencies.


In [39]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler(feature_range=(0,1))
scaler_cluster = scaler.fit_transform(clusters_df[["TotalCoinSupply", "TotalCoinsMined"]])
scaler_cluster

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [40]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_df = pd.DataFrame(scaler_cluster, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clusters_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_df = pd.DataFrame({
    "TotalCoinSupply": scaled_df["TotalCoinSupply"],
    "TotalCoinsMined": scaled_df["TotalCoinsMined"],
    "CoinName": clusters_df["CoinName"],
    "Class": clusters_df["Class"]
})


scaled_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [41]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaled_df.hvplot(
    kind="scatter",
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    by="Class",
    hover_cols=["CoinName"]
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)